# Compress τ dependence

In [ ]:
using Revise
using Plots
using LinearAlgebra
using SparseIR
import SparseIR: valueim

newaxis = [CartesianIndex()]

In [ ]:
BLAS.set_num_threads(16)

In [ ]:
println(Threads.nthreads())

In [ ]:
using ITensors

println(ITensors.blas_get_num_threads())

In [ ]:
beta = 10.0
wmax = 10.0

basis = SparseIR.FiniteTempBasis(Fermionic(), beta, wmax, 1e-10)
smpl = SparseIR.MatsubaraSampling(basis)
;

In [ ]:
# Three Gaussian peaks (normalized to 1)
gaussian(x, mu, sigma) = exp(-((x-mu)/sigma)^2)/(sqrt(π)*sigma)

rho(omega) = 0.2*gaussian(omega, 0.0, 0.15) + 
    0.4*gaussian(omega, 1.0, 0.8) + 0.4*gaussian(omega, -1.0, 0.8)

omegas = LinRange(-5, 5, 1000)
plot(omegas, rho.(omegas), label="")

In [ ]:
rhol = [overlap(basis.v[l], rho) for l in 1:length(basis)]
gl = - basis.s .* rhol

plot([abs.(rhol), abs.(gl)], yaxis=:log, ylims=(1e-5,1), marker=[:circle :diamond], line=nothing)

In [ ]:
N = 12
half_N = N ÷ 2
ntau = 2^N
half_ntau = 2^(N-1)
ntau

In [ ]:
taus_half = collect(LinRange(0, beta, half_ntau+1)[1:end-1])
taus = collect(LinRange(0, 2*beta, ntau+1))[1:end-1]
gtau_half = evaluate(TauSampling(basis, taus_half), gl)
gtau = [gtau_half; -gtau_half] 
;

In [ ]:
plot(taus, real.(gtau), marker=:x)

In [ ]:
tensor = reshape(gtau, repeat([2,], N)...)
;

In [ ]:
u, s, vt = svd(reshape(tensor, :, 2^half_N))
;

In [ ]:
plot(s./s[1], yaxis=:log)

In [ ]:
sites = siteinds(2, N)

In [ ]:
cutoff = 1E-20
maxdim = 20
M = MPS(tensor, sites; cutoff=cutoff, maxdim=maxdim)

In [ ]:
tensor_reconst = Array(reduce(*, M), sites...)
;

In [ ]:
plot(abs.(vec(tensor_reconst) .- vec(tensor)))

In [ ]:
bonddims = collect(size(m)[1] for m in M)
plot(bonddims, marker=:x)